In [57]:
import pandas as pd
import numpy as np

df = pd.read_csv("training_data_customers.csv")
final_df = df.drop(columns="Unnamed: 1")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

## Random Forest
# Best parameters found
best_params = {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}

# Create and train the Random Forest model with the best parameters
rf_model = RandomForestRegressor(random_state=42, **best_params)  # **best_params unpacks the dictionary

In [ ]:
#Update Customer Data
def update_customer_data(new_customer_counts):
    """Updates customer data for shifts 1, 2, and 3 with rounded values.

    Args:
        new_customer_counts: A NumPy array or a list/tuple that can be converted
                             to a NumPy array of customer counts.

    Returns:
        The updated DataFrame.
    """

    global final_df

    new_customer_counts = np.array(new_customer_counts) # Ensure it's a NumPy array
    if len(new_customer_counts) != 3:
        raise ValueError("new_customer_counts must be a list or tuple of length 3.")


    rounded_counts = np.round(new_customer_counts).astype(int) # Round and convert to integers

    last_group = final_df['Group'].max()
    if pd.isna(last_group):
        new_group = 1
    else:
        new_group = last_group + 1

    new_rows = []
    for i, customers in enumerate(rounded_counts):
        new_rows.append({'Group': new_group, 'Shift': i + 1, 'Customers': customers})

    final_df = pd.concat([final_df, pd.DataFrame(new_rows)], ignore_index=True)
    return final_df

In [60]:
#Retraining the model
def retraining(final_df):
    """Retrains the model with the updated DataFrame.

    Args:
        final_df: The updated pandas DataFrame.

    Returns:
        The retrained model (it's generally better to return the model itself).
    """
    X = final_df[['Shift']]
    y = final_df['Customers']

    rf_model.fit(X, y)  # Retrain the model

    return rf_model

In [61]:
# Retrain the model:
retrained_model = retraining(final_df)  # Store the retrained model

#Now you can use predict then retrained the model:
customer_data = np.array([[1], [2], [3]])
new_predictions = retrained_model.predict(customer_data)
final_df = update_customer_data(new_predictions)

c:\Users\yuliastu\Music\Feni\Uni\Master Dokumen\Intelligent Agent\mesa-restaurant-agents\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
import mesa

# Data (should be accessible to the model)
shifts = [1, 2, 3]
demands = {1: 40, 2: 45, 3: 60}
n_fulltime = 3
n_parttime = 2
capacity = 20
fulltime_names = ["Ana", "Bob", "Alice"]
parttime_names = ["Laura", "Bill"]

class Waiter(mesa.Agent):
    def __init__(self, unique_id, model, is_fulltime):
        super().__init__(unique_id, model)
        self.is_fulltime = is_fulltime
        self.shifts_worked = 0  # Track shifts worked
        self.available = True

    def work_shift(self):
      self.shifts_worked += 1
      self.available = False # Waiter is no longer available in this day

    def reset_availability(self):
      self.available = True
      if self.is_fulltime and self.shifts_worked > 2:
        self.available = False # Full time waiter can only work max 2 shifts
      elif not self.is_fulltime and self.shifts_worked > 1:
        self.available = False # Part time waiter can only work max 1 shift


class RestaurantModel(mesa.Model):
    def __init__(self, num_waiters):
        super().__init__()
        self.num_waiters = num_waiters
        self.waiters = [] # List to hold waiter agents
        self.schedule = mesa.time.SequentialScheduler(self) # Run all agent in the same time
        self.final_df = pd.DataFrame(columns=['Group', 'Shift', 'Customers']) # DataFrame for training
        self.group_counter = 1

        # Create waiters
        waiter_id = 0
        for _ in range(n_fulltime):
            self.waiters.append(Waiter(waiter_id, self, True))
            waiter_id += 1
        for _ in range(n_parttime):
            self.waiters.append(Waiter(waiter_id, self, False))
            waiter_id += 1


        for waiter in self.waiters:
            self.schedule.add(waiter)

    def step(self):
        # 1. Prediction and Retraining
        self.retrain_and_predict()

        # 2. Assign waiters to shifts
        self.assign_waiters()

        # 3. Reset waiter availability for the next day
        for waiter in self.waiters:
            waiter.reset_availability()

    def retrain_and_predict(self):
        if not self.final_df.empty:  # Only retrain if there's data
            retrained_model = retraining(self.final_df)  # Assuming retraining is defined
            customer_data = np.array([[self.group_counter]]) # Create new group for prediction
            new_predictions = retrained_model.predict(customer_data)
            rounded_predictions = np.round(new_predictions).astype(int)
            new_data = {'Group': self.group_counter, 'Shift': 1, 'Customers': rounded_predictions[0]} # Only predict for shift 1, because demand for shift 2 and 3 is fixed.
            self.final_df = pd.concat([self.final_df, pd.DataFrame([new_data])], ignore_index=True)
            self.group_counter += 1

    def assign_waiters(self):
        for shift in shifts:
            demand = demands[shift]
            total_capacity_covered = 0
            required_waiters = 0

            available_waiters = [waiter for waiter in self.waiters if waiter.available]

            if len(available_waiters) < 2:
                raise Exception(f"Not enough waiters available for shift {shift}")

            # Assign waiters to the shift
            for waiter in available_waiters:
                if total_capacity_covered < demand:
                    waiter.work_shift()
                    total_capacity_covered += capacity
                    required_waiters += 1
                else:
                  break # Demand is covered

            if total_capacity_covered < demand:
                raise Exception(f"Not enough capacity covered for shift {shift}")

            print(f"Shift {shift}: {required_waiters} waiters assigned (Demand: {demand}, Capacity Covered: {total_capacity_covered})")

c:\Users\yuliastu\Music\Feni\Uni\Master Dokumen\Intelligent Agent\mesa-restaurant-agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
